In [7]:
import matplotlib.pyplot as plt

In [19]:
from ucimlrepo import fetch_ucirepo
  
# fetch dataset 
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17) 
  
# data (as pandas dataframes) 
X = breast_cancer_wisconsin_diagnostic.data.features 
y = breast_cancer_wisconsin_diagnostic.data.targets 

# metadata
# print(breast_cancer_wisconsin_diagnostic.metadata)
# Diagnosis (M = malignant, B = benign)
  
# variable information 
# print(breast_cancer_wisconsin_diagnostic.variables) 

In [10]:
training_X = X[:248]
training_y = y[:248]

testing_X = X[248:]
testing_y = y[248:]